# Web Scraping for Indeed.com & Predicting Salaries

In this project, I will practice two major skills: collecting data by scraping a website and then building a binary classifier.

I am going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job I will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information, being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

I will convert this problem into classification and use a random forest classifier, as well as another classifier. 

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

I will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
from urllib import urlopen
import pandas as pd
import matplotlib as plt
%matplotlib inline

In [3]:
## YOUR CODE HERE
html = urllib.urlopen(URL).read()
soup = BeautifulSoup(html, "html.parser")
len(soup)

4

Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Functions to extract each item: location, company, job, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```


In [4]:
dflocation = pd.DataFrame(columns=["location"])
dfcompany = pd.DataFrame(columns=["company"])
dfjob_title = pd.DataFrame(columns=["job_title"])
dfsalary = pd.DataFrame(columns=["salary"])

#result.find(‘a’,attrs={‘data-tn-element’:‘jobTitle’}).text
def extract_location(result):
    for b in result.find_all('span', {'class': 'location'}):
        location = b.text
        dflocation.loc[len(dflocation)] = [location]    
        
def extract_company(result):        
    for i in result.find_all('span', {'class':'company'}):
        company = i.text
        dfcompany.loc[len(dfcompany)] = [company]   

def extract_job_title(result):
    for a in result.find_all('a', {'data-tn-element':'jobTitle'}):
        job_title = a.text
        dfjob_title.loc[len(dfjob_title)] = [job_title]

def extract_salary(result):
    for entry in result.find_all('td', {'class' : 'snip'}):
        try:
            salary = entry.find('nobr').text
            dfsalary.loc[len(dfsalary)] = [salary]  
        except:
            salary = 'NA'
            dfsalary.loc[len(dfsalary)] = [salary]    


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).


In [5]:
#making a list of cities to run through
cities = ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'washington+dc', 'boston', 'new+orleans', 
         'charlotte']

In [9]:
#making a dataframe which collects all the information from my webscraping
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value to generate more results. 

df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary'])

for city in cities:
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        html = urllib.urlopen(url).read()
        soups = BeautifulSoup(html, "html.parser")
        for b in soups.find_all('div', attrs = {'class':' row result'}):
            location = b.find('span', attrs = {'class': 'location'}).text
            job_title = b.find('a', attrs = {'data-tn-element':'jobTitle'}).text
            try:
                company = b.find('span', attrs = {'class':'company'}).text
            except:
                company = 'NA'
            try:
                salary = b.find('td', attrs = {'class' : 'snip'}).find('nobr').text  
            except:
                salary = 'NA'
            df = df.append({"location":location,"company":company, "job_title": job_title, "salary": salary}, ignore_index=True) 
   
df.head()        

,location,company,job_title,salary
0,"New York, NY",\n\n\n IBM\n,Data Scientist,NA
1,"New York, NY",\n\n White Ops\n,Data Scientist,NA
2,"New York, NY 10017 (Midtown area)",\n\n\n Guidepoint\n,Data Scientist,NA
3,"New York, NY 10005 (Financial District area)",\n\n Enterprise Select\n,Data Scientist,NA
4,"New York, NY",\n\n Innovations for Poverty\n,Data Scientist,NA


In [ ]:
#sending it to csvs to save the data
#df.to_csv("~/Desktop/April12.csv" , sep=',', encoding='utf-8')

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [11]:
#reading in the data
df1 = pd.read_csv('~/Desktop/GA/April10.csv')
df2 = pd.read_csv('~/Desktop/GA/April11.csv')
df3 = pd.read_csv('~/Desktop/GA/April12.csv')
df4 = pd.read_csv('~/Desktop/GA/April1_11.csv')
df5 = pd.read_csv('~/Desktop/GA/April2_11.csv')
df6 = pd.read_csv('~/Desktop/GA/April_10.csv')
df7 = pd.read_csv('~/Desktop/GA/April_11.csv')

In [12]:
data = pd.concat([df1,df2,df3,df4,df5,df6,df7]) #making into one df
data.drop(['Unnamed: 0'], axis=1, inplace=True) #resetting index
data.drop_duplicates(inplace=True) #dropping duplicates
data.company.replace(regex=True,inplace=True,to_replace="\n",value="") #getting rid of /n in company
data.salary.replace(regex=True, inplace=True, to_replace="\$", value="") #getting rid of $ in salary

In [13]:
def eda(dataframe): #code chunk to check quality of data
    print "missing values \n", dataframe.isnull().sum() #shows total amount of null values for each column
    print "dataframe types \n", dataframe.dtypes
    print "dataframe shape \n", dataframe.shape     
    print "dataframe describe \n", dataframe.describe()
    print "dataframe length =", len(dataframe) #length of the dataframe
    print "duplicates", dataframe.duplicated().sum() # added this to duplicates in the data
    for item in dataframe:
        print item
        print dataframe[item].nunique()

eda(data)

missing values 
location        0
company         3
job_title       0
salary       8134
dtype: int64
dataframe types 
location     object
company      object
job_title    object
salary       object
dtype: object
dataframe shape 
(8698, 4)
dataframe describe 
            location                 company       job_title          salary
count           8698                    8695            8698             564
unique           935                    3076            6498             334
top     New York, NY          Ball Aerospace  Data Scientist  120,000 a year
freq             653                     126             355              13
dataframe length = 8698
duplicates 0
location
935
company
3076
job_title
6498
salary
334


Lastly, I need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

I didn't think it was safe to multiply data for hour/week/month so I only wanted yearly data.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly, remove duplicate entries

In [14]:
salaries = data[data.salary.notnull()]
salaries = salaries[salaries.salary.str.contains('year')] #only getting yearly salaries
#http://stackoverflow.com/questions/15325182/how-to-filter-rows-in-pandas-by-regex
salaries.describe()

,location,company,job_title,salary
count,402,401,402,402
unique,122,197,317,219
top,"New York, NY",Jobspring Partners,Data Scientist,"120,000 a year"
freq,59,25,26,13


In [15]:
#splitting my salaries to find an average
salaries.salary.replace(regex=True, inplace=True, to_replace="a year", value="")
salaries.salary.replace(regex=True, inplace=True, to_replace=",", value="")
salaries['salary_split'] = salaries['salary'].str.split('-')
salaries.head()

,location,company,job_title,salary,salary_split
1,"New York, NY 10005 (Financial District area)",Barrington James,Real World Evidence Statistical Analyst,110000,[110000 ]
5,"New York, NY",PMES,Sr Data Scientist,155000,[155000 ]
6,"New York, NY",indify,Data Scientist,90000 - 170000,"[90000 , 170000 ]"
27,"New York, NY",WorldCover,Data Scientist,70000 - 110000,"[70000 , 110000 ]"
51,"New York, NY","Envisagenics, Inc.",Data Architect/ Data Engineer,65000 - 110000,"[65000 , 110000 ]"


In [19]:
#finding an average salary
def avg(salaries):
    salaries['lower'] = salaries['salary_split'].str[0].astype('float')
    salaries['upper'] = salaries['salary_split'].str[1].astype('float')
    salaries['avg'] = salaries[['lower','upper']].mean(axis=1)
avg(salaries)
salaries.head()

,location,company,job_title,salary,salary_split,lower,upper,avg
1,"New York, NY 10005 (Financial District area)",Barrington James,Real World Evidence Statistical Analyst,110000,[110000 ],110000.0,NaN,110000.0
5,"New York, NY",PMES,Sr Data Scientist,155000,[155000 ],155000.0,NaN,155000.0
6,"New York, NY",indify,Data Scientist,90000 - 170000,"[90000 , 170000 ]",90000.0,170000.0,130000.0
27,"New York, NY",WorldCover,Data Scientist,70000 - 110000,"[70000 , 110000 ]",70000.0,110000.0,90000.0
51,"New York, NY","Envisagenics, Inc.",Data Architect/ Data Engineer,65000 - 110000,"[65000 , 110000 ]",65000.0,110000.0,87500.0


In [20]:
#dropping columns so I'm only left with average column
clean_sal = salaries.drop(['salary','salary_split', 'lower', 'upper'], axis=1)
clean_sal.head()

,location,company,job_title,avg
1,"New York, NY 10005 (Financial District area)",Barrington James,Real World Evidence Statistical Analyst,110000.0
5,"New York, NY",PMES,Sr Data Scientist,155000.0
6,"New York, NY",indify,Data Scientist,130000.0
27,"New York, NY",WorldCover,Data Scientist,90000.0
51,"New York, NY","Envisagenics, Inc.",Data Architect/ Data Engineer,87500.0


In [21]:
clean_sal['citystate'] = clean_sal['location'].str.split(',') #splitting the location to separate city and state
clean_sal['city'] = clean_sal['citystate'].str[0] #getting cities
clean_sal['state'] = clean_sal['citystate'].str[1] #getting states
clean_sal['state'] = clean_sal['state'].str[0:3] #getting only 2 letter state codes
clean_sal.drop(['location','citystate'], axis=1, inplace=True) #dropping columns so I'm only left with cities/states
clean_sal.head()

,company,job_title,avg,city,state
1,Barrington James,Real World Evidence Statistical Analyst,110000.0,New York,NY
5,PMES,Sr Data Scientist,155000.0,New York,NY
6,indify,Data Scientist,130000.0,New York,NY
27,WorldCover,Data Scientist,90000.0,New York,NY
51,"Envisagenics, Inc.",Data Architect/ Data Engineer,87500.0,New York,NY


In [22]:
eda(clean_sal)

missing values 
company      1
job_title    0
avg          0
city         0
state        0
dtype: int64
dataframe types 
company       object
job_title     object
avg          float64
city          object
state         object
dtype: object
dataframe shape 
(402, 5)
dataframe describe 
                 avg
count     402.000000
mean   111915.820896
std     49224.798792
min     24000.000000
25%     75000.000000
50%    106250.000000
75%    144887.500000
max    275000.000000
dataframe length = 402
duplicates 0
company
197
job_title
317
avg
148
city
72
state
18


### Save your results as a CSV

In [23]:
#saving my clean salary data to a csv
clean_sal.to_csv("~/Desktop/clean_salary.csv" , sep=',', encoding='utf-8')

## Predicting salaries using Random Forests + Another Classifier

In [24]:
clean_sal['avg'].describe()

count       402.000000
mean     111915.820896
std       49224.798792
min       24000.000000
25%       75000.000000
50%      106250.000000
75%      144887.500000
max      275000.000000
Name: avg, dtype: float64

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [25]:
#finding the median
import numpy as np
median = np.median(clean_sal.avg)
print median

106250.0


In [27]:
c = pd.read_csv('~/Desktop/GA/clean_sal.csv')

In [28]:
c['dumsal'] = (c["avg"] >= c["avg"].median()).astype(int)
#http://stackoverflow.com/questions/36637011/how-can-i-create-a-dummy-variable-in-python-with-a-condition-below-or-above-medi
c.head()

,Unnamed: 0,company,job_title,avg,city,state,dumsal
0,1,Barrington James,Real World Evidence Statistical Analyst,110000.0,New York,NY,1
1,5,PMES,Sr Data Scientist,155000.0,New York,NY,1
2,6,indify,Data Scientist,130000.0,New York,NY,1
3,27,WorldCover,Data Scientist,90000.0,New York,NY,0
4,51,"Envisagenics, Inc.",Data Architect/ Data Engineer,87500.0,New York,NY,0


In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()
X = c.job_title
y = c.dumsal
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [31]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_train = pd.DataFrame(cvec.transform(X_train).todense(),
             columns=cvec.get_feature_names())

In [32]:
df_test = pd.DataFrame(cvec.transform(X_test).todense(),
                      columns=cvec.get_feature_names())
print df_train.shape
print y_train.shape
print df_test.shape
print y_test.shape

(269, 314)
(269,)
(133, 314)
(133,)


In [33]:
lr = LogisticRegression()
lr.fit(df_train, y_train)
lr.score(df_test, y_test)
#this is my title score

0.80451127819548873

In [44]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
# Perform 6-fold cross validation
scores = cross_val_score(lr, df_test, y_test, cv=6)
print "Cross-validated scores:", scores
# Make cross validated predictions
predictions = cross_val_predict(lr, df_test, y_test, cv=6)
accuracy = metrics.accuracy_score(y_test, predictions)
print "Cross-Predicted Accuracy:", accuracy

Cross-validated scores: [ 0.47826087  0.39130435  0.40909091  0.40909091  0.36363636  0.38095238]
Cross-Predicted Accuracy: 0.406015037594


In [37]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight='balanced')
RF = rf.fit(df_train,y_train)
s = cross_val_score(rf, df_train, y_train, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forrest", s.mean().round(3), s.std().round(3))

Random Forrest Score:	0.807 ± 0.01


In [39]:
rf_features = pd.DataFrame(RF.feature_importances_,
                                   index = df_train.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
rf_features.head(10)

,importance
data,0.104554
scientist,0.053279
learning,0.052167
analyst,0.037202
engineer,0.031317
machine,0.026832
research,0.022557
senior,0.021921
science,0.021450
analytics,0.018560


#### Create a few new variables in my dataframe to represent interesting features of a job title. Then I built a new Random Forest with these features. 

In [40]:
#manager, visualization, scientist, engineer, analyst, senior
c['visualization'] = map(int, c['job_title'].str.lower().str.contains('vis'))
c['manager'] = map(int, c['job_title'].str.lower().str.contains('manager')|
                  c['job_title'].str.lower().str.contains('mngr')|
                  c['job_title'].str.lower().str.contains('mgr'))
c['scientist'] = map(int, c['job_title'].str.lower().str.contains('scien'))
c['engineer'] = map(int, c['job_title'].str.lower().str.contains('eng'))
c['senior'] = map(int, c['job_title'].str.lower().str.contains('sr')|
                  c['job_title'].str.lower().str.contains('snr')|
                  c['job_title'].str.lower().str.contains('senior'))
#making dummies for the words at the top of this cell
c.head()

,Unnamed: 0,company,job_title,avg,city,state,dumsal,visualization,manager,scientist,engineer,senior
0,1,Barrington James,Real World Evidence Statistical Analyst,110000.0,New York,NY,1,0,0,0,0,0
1,5,PMES,Sr Data Scientist,155000.0,New York,NY,1,0,0,1,0,1
2,6,indify,Data Scientist,130000.0,New York,NY,1,0,0,1,0,0
3,27,WorldCover,Data Scientist,90000.0,New York,NY,0,0,0,1,0,0
4,51,"Envisagenics, Inc.",Data Architect/ Data Engineer,87500.0,New York,NY,0,0,0,0,1,0


In [41]:
#completing a logistic regression model
from sklearn.linear_model import LogisticRegression
X = c[['manager', 'visualization', 'scientist', 'engineer', 'senior']]
y = c.dumsal
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)
#this is my concatonated score since it's worse I'm only going to use  

0.76691729323308266

In [42]:
#and a random forest classifier
rf = RandomForestClassifier(class_weight='balanced')
s = cross_val_score(rf, X, y, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forrest", s.mean().round(3), s.std().round(3))

Random Forrest Score:	0.729 ± 0.019


In [43]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
# Perform 6-fold cross validation
scores = cross_val_score(rf, X, y, cv=6)
print "Cross-validated scores:", scores
# Make cross validated predictions
predictions = cross_val_predict(rf, X, y, cv=6)
accuracy = metrics.accuracy_score(y, predictions)
print "Cross-Predicted Accuracy:", accuracy

Cross-validated scores: [ 0.58823529  0.77941176  0.79411765  0.74242424  0.68181818  0.75757576]
Cross-Predicted Accuracy: 0.713930348259


In [38]:
#sending my final to a csv
c.to_csv("~/Desktop/c.csv" , sep=',', encoding='utf-8')